In [2]:
#!pip install -q sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as  np


print("Libraries imported successfully!1")


Libraries imported successfully!1


In [4]:
# Loaad a small, fast embedding model
print("Loading embedding model...")

model = SentenceTransformer ('all-MiniLM-L6-v2')
print("Model loaded!")
print(f"Model produces {model.get_sentence_embedding_dimension()} dimensional embeddings")


Loading embedding model...
Model loaded!
Model produces 384 dimensional embeddings


Generate Embeddings

In [5]:
# Simple example

text = "The cat sat on the met"

# Generate embedding
embedding = model.encode(text)

print(f"Original text: {text}")
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding type: {type(embedding)}")
print(f"\nFirst 10 values: {embedding[:10]}")

Original text: The cat sat on the met
Embedding shape: (384,)
Embedding type: <class 'numpy.ndarray'>

First 10 values: [ 0.07994752 -0.00066558  0.01172572  0.08463687 -0.10899851  0.02930527
  0.01828371  0.00536318 -0.03327654 -0.01049046]


Similarity: The Heart of RAG

Cosine Similarity Explained
cosine similarity measures how similar two vectors are.




In [6]:
def cosine_similarity(vec1, vec2):
    """
    calculate cosine similarity between two vectors
    
    Returns a score between -1 and 1 (higher = more similar)
    """
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product /  (norm1 * norm2)

print(" Similarity function ready!")

 Similarity function ready!


Testing Similarity

In [7]:
# Create test sentences
sentences = [
    "The cat sat on the mat",
    "A feline rested on the rug",      # Similar meaning, different word
    "Dogs are loyal animals",           # Differnt topic
    "Python is a programming language"   #completely unrelated
]


# Generate embeddings for all sentences
embeddings = model.encode(sentences)

# Compare first sentence to all others
print("Comparing to: 'The cat sat on the mat'\n")
for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[0], embeddings[i])
    print(f"Similarity to '{sentence}'")
    print(f"Score: {similarity:.3f}\n")

Comparing to: 'The cat sat on the mat'

Similarity to 'The cat sat on the mat'
Score: 1.000

Similarity to 'A feline rested on the rug'
Score: 0.564

Similarity to 'Dogs are loyal animals'
Score: 0.165

Similarity to 'Python is a programming language'
Score: 0.031



Building a Simple Semantic Search

Create a Document Collection

In [8]:
# Sample Knowledge base
documents = [
    "Python is high-level programming language known for simplicity",
    "Machine learning enables computers to learn from data",
    "Neural networks are inspired by biological brains",
    "Dogs are loyal and friendly pets that need exercise",
    "Cats are independent animals that make grat combinations",
    "JavaScript is used for web development and runs in browsers",
    "Deep learning uses multi_layered neural networks",
    "Puppies require training and socialization from an early age"
]

print(f"Knowlege base: {len(documents)} documents")

Knowlege base: 8 documents


Embed All Documents

In [9]:
# Generate embeddings for all documents

print("Generating embeddings for all documents....")
doc_embeddings = model.encode(documents)

print(f" Created {len(doc_embeddings)} embeddings")
print(f"Each embeddings has {doc_embeddings[0].shape[0]} dimensions")


Generating embeddings for all documents....
 Created 8 embeddings
Each embeddings has 384 dimensions


Search Function

In [10]:
def search(query, documents, doc_embeddings, top_k=3):
    """
    Search for documents similar to the query.


    Args:
        query: Search (string)
        documents: LIst of document texts
        doc_embeddings: Pre-computed document embeddings
        top_k: Number of results to return

    Returns:
        Lists of  (document, similarity_score) tuples
    """

    # Embed the query
    query_embedding = model.encode(query)

    # Calculate similarities
    similarities = []
    for i, doc_emb in enumerate(doc_embeddings):
        similarity = cosine_similarity(query_embedding, doc_emb)
        similarities.append((documents[i], similarity))

    # sort by similarity (highest first)
    similarities.sort(key=lambda x:x[1], reverse=True)
 

    # Return top k results
    return similarities[:top_k]


print("Search function ready!")

Search function ready!


## Continuation on Module8

Build complete RAG systems using LangChain's modern LCEL (LangChain Expression Language)


Setup
Require packages

pip install langchain langchain-core langchain-community langchain-openai langchain-text-splitters

pip install faiss-cpu python-dotenv


In [11]:
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_community.document_loaders import PyPDFLoader
import os
from dotenv import load_dotenv

load_dotenv()
api_key= os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print('ApI key loaded')



ValueError: OPENAI_API_KEY not found in .env file

In [13]:
from langchain_core.documents import Document


# create documents directly (you can also load from files using loaders)
documents = [

     Document(page_content="LangChain is a framework for developing applications powered by language models."),
    Document(page_content="RAG combines retrieval with generation to provide accurate, grounded responses."),
    Document(page_content="Vector stores enable efficient similarity search over embedded documents."),
    Document(page_content="FAISS is a library for efficient similarity search developed by Meta AI."),
    Document(page_content="Text splitters chunk documents into smaller pieces for better retrieval.")
]

print(f"Created {len(documents)} documents")
print(f'First document: {documents[0].page_content}')

Created 5 documents
First document: LangChain is a framework for developing applications powered by language models.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)


# Split documents
chunks =text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}: {chunk.page_content}")

    chunks= text_splitter.split_documents(documents)